In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
Proj_dir=r"D:\ML_Project"
Data_dir=r"Data"

In [3]:
os.path.join(Proj_dir,Data_dir,"flight_price.csv")

'D:\\ML_Project\\Data\\flight_price.csv'

In [4]:
def get_data(name):
    file_name=f"{name}.csv"
    file_path=os.path.join(Proj_dir,Data_dir,file_name)
    return pd.read_csv(file_path)

In [5]:
flights=get_data("flight_price")

In [6]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [7]:
#10683 rows
#cols route and total stops have missing values
#datatypes of some features isnt appropriate


#Date of journey, Dep_Time, Arrival_Time is string type, need to be date-time
#duration to just minutes instead of 2h 50m seperate seperate
#220 duplicate rows

In [8]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [9]:
flights.Date_of_Journey.iloc[1]

'1/05/2019'

In [10]:
flights.Source[1]

'Kolkata'

In [11]:
flights.Route.iloc[1]

'CCU → IXR → BBI → BLR'

In [12]:
flights.Dep_Time.iloc[1]

'05:50'

In [13]:
flights.Arrival_Time.iloc[1]

'13:15'

In [14]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [15]:
flights.duplicated().sum()

np.int64(220)

In [16]:
flights[flights.duplicated()].sort_values(["Airline","Date_of_Journey"])

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
9913,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
5042,Air India,1/05/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,13:45 02 May,27h 45m,2 stops,No info,15164
6377,Air India,1/05/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,13:45 02 May,27h 45m,2 stops,No info,15164
...,...,...,...,...,...,...,...,...,...,...,...
9216,Jet Airways,9/06/2019,Delhi,Cochin,DEL → IDR → BOM → COK,05:30,04:25 10 Jun,22h 55m,2 stops,No info,13292
7502,SpiceJet,03/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No info,6860
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873


In [17]:
##Cleaning

In [18]:
#Flights
flights.Airline.unique()                #some entries inaccurate

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [19]:
(
    flights
    .Airline
    .str.replace(" Premium economy","")
    .str.replace(" Business","")                          #experimenting
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

In [20]:
#Date_of_Journey
flights.Date_of_Journey             #dtype need to be changed to date time

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [21]:
pd.to_datetime(flights.Date_of_Journey,dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

In [22]:
#Source
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [23]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [24]:
#Route is a redundant feature, so lets drop it

In [25]:
#Dep_Time          hr:min        how to check if no special chars present in string
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [26]:
(
    flights
    .Dep_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]             # ^ give us any row where those characters are not present
)                                                            #gives empty series, that is, it is perfect

Series([], Name: Dep_Time, dtype: object)

In [27]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\Dhanj\AppData\Local\Temp\ipykernel_2664\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [28]:
#Arrival time
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [29]:
(
    flights
    .Arrival_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]
    .str.split(" ",n=1)
    .str.get(0)
    .unique()
)

array(['01:10', '04:25', '10:25', '05:05', '09:20', '19:15', '12:35',
       '13:20', '08:15', '03:35', '19:00', '11:05', '10:55', '00:15',
       '02:45', '12:00', '01:30', '16:45', '09:25', '04:40', '11:25',
       '14:25', '14:35', '09:05', '01:35', '10:05', '11:10', '07:45',
       '00:55', '23:35', '00:40', '18:50', '07:55', '18:15', '13:15',
       '09:45', '06:50', '15:15', '07:40', '02:20', '18:30', '19:45',
       '19:50', '01:00', '00:05', '16:20', '00:30', '00:50', '13:45',
       '09:15', '07:10', '08:55', '04:45', '05:35', '09:00', '10:45',
       '22:10', '22:40', '05:25', '02:15', '20:20', '06:35', '01:20',
       '19:10', '10:50', '23:25', '12:15', '10:40', '23:15', '22:55',
       '09:10', '18:10', '00:25', '20:45', '21:05', '23:00', '00:45',
       '04:35', '18:00', '12:30', '02:10', '00:20', '23:20', '12:25',
       '22:35', '20:25', '00:35', '16:50', '09:30', '12:05', '10:35',
       '15:05', '18:40', '09:35', '08:50', '16:40', '19:40', '16:10',
       '21:20', '07:

In [30]:
#Duration
(
    flights
    .Duration
    .loc[lambda ser:~ser.str.contains("m")]
)

2        19h
18       23h
33       22h
44       12h
53        3h
        ... 
10591    23h
10638    14h
10639    38h
10673    15h
10680     3h
Name: Duration, Length: 1031, dtype: object

In [31]:
(
    flights
    .Duration
    .loc[lambda ser:~ser.str.contains("h")]
)

6474    5m
Name: Duration, dtype: object

In [32]:
flights.iloc[[6474]]                           #not a valid entry
                                               #6474 has 5m duration, hence needs to be deleted

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [33]:
(
    flights
    .Duration
    .drop(index=[6474])
    .str.split(" ",expand=True)                #expand=True gives dataframe
    .set_axis(["hour","minute"],axis=1)        #quickly renames
    .assign(
        hour=lambda df_:(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)                         #pandas method for multiplication, similar for add subb etc
        ),
        minute=lambda df:(
            df
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    #.dtypes
    #.isna().sum()
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [34]:
(
    flights
    .Duration
    .drop(index=[6474])
    .str.split(" ",expand=True)                #expand=True gives dataframe
    .set_axis(["hour","minute"],axis=1)        #quickly renames
    .assign(
        hour=lambda df_:(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)                         #pandas method for multiplication, similar for add subb etc
        ),
        minute=lambda df:(
            df
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    #.dtypes
    #.isna().sum()
    .rename("duration_minutes")
    .to_frame()
    .join(flights.Duration.drop(index=[6474]))                                       #for side by side comparison
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


In [35]:
#Total Stops
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [36]:
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","",regex=True)           # ? means s is optional now cos of regex
    .pipe(lambda ser:pd.to_numeric(ser))            #converts to float                  #cant do astype int due to missing data
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

In [37]:
#Additional Info
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [38]:
##############   CLEANING    ################
def convert_to_minutes(ser):
    return(
    ser
    .str.split(" ",expand=True)                #expand=True gives dataframe
    .set_axis(["hour","minute"],axis=1)        #quickly renames
    .assign(
        hour=lambda df_:(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)                         #pandas method for multiplication, similar for add subb etc
        ),
        minute=lambda df:(
            df
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    )

In [39]:

def clean_data(df):
    return( 
        df
        .drop(index=[6474])
        .drop_duplicates()
        .assign(
            # airline=lambda df_:df_.airline.str.lower()    #method to create and update columns
            **{col:df[col].str.strip() for col in df.select_dtypes(include="O").columns}      #selects all cols with dtype object
                                                                                                         # ** unpacks dict, col name=operation
        )   
        .rename(columns=str.lower)                        #change doesnt effect original object
                                                          #df_ is just a convention, the current state of df is passed into it
        .assign(
            airline=lambda df_:(
                df_
                .airline
                .str.replace(" Premium economy","")
                .str.replace(" Business","")
                .str.title()
            ),
            date_of_journey=lambda df_:pd.to_datetime(df_.date_of_journey,dayfirst=True),
            
            dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
            
            arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,
            
            #duration=lambda df_:convert_to_minutes(df_.duration)             #works but better way is with pipe
            
            duration=lambda df_:df_.duration.pipe(convert_to_minutes),         #pipe called on ser or df, fn in it should accept a series or df, ret anything
            
            total_stops=lambda df_:(
                df_
                .total_stops
                .replace("non-stop","0")
                .str.replace(" stops?","",regex=True)           
                .pipe(lambda ser:pd.to_numeric(ser))
            ),
            additional_info=lambda df_:df_.additional_info.replace("No info","No Info")
        )
        .drop(columns="route")
                             
    )

In [40]:
flights_cleaned=clean_data(flights)
flights_cleaned

C:\Users\Dhanj\AppData\Local\Temp\ipykernel_2664\2301020096.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
C:\Users\Dhanj\AppData\Local\Temp\ipykernel_2664\2301020096.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


In [41]:
flights.select_dtypes(include="O")

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info


In [47]:
#Split the data  #10000 rows takes too much time, hence taking random sample of 1000

In [48]:
flights_final=flights_cleaned.sample(1000)
X=flights_final.drop(columns="price")
y=flights_final.price.copy()

In [49]:
X_,X_test,y_,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_,y_,test_size=0.2,random_state=42)

In [50]:
print(X_train.shape,y_train.shape)

(640, 9) (640,)


In [51]:
print(X_val.shape,y_val.shape)

(160, 9) (160,)


In [52]:
print(X_test.shape,y_test.shape)

(200, 9) (200,)


In [62]:
#Exporting the Data

def export_data(X,y,name):
    file_name=f"{name}.csv"
    file_path=os.path.join(Proj_dir,Data_dir,file_name)
    X.join(y).to_csv(file_path,index=False)
    return pd.read_csv(file_path).head()

In [63]:
#export_data(X_train,y_train,"train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-06-09,Kolkata,Banglore,16:50:00,05:35:00,765,2.0,No Info,11642
1,Goair,2019-06-15,Banglore,Delhi,20:55:00,23:40:00,165,0.0,No Info,3898
2,Multiple Carriers,2019-03-21,Delhi,Cochin,10:00:00,21:00:00,660,1.0,In-flight meal not included,8085
3,Jet Airways,2019-05-09,Delhi,Cochin,05:30:00,04:25:00,1375,2.0,In-flight meal not included,13029
4,Indigo,2019-04-18,Kolkata,Banglore,21:25:00,00:05:00,160,0.0,No Info,4174


In [64]:
#export_data(X_val,y_val,"val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Spicejet,2019-04-12,Kolkata,Banglore,06:55:00,09:30:00,155,0.0,No check-in baggage included,3841
1,Air India,2019-03-27,Mumbai,Hyderabad,13:55:00,15:25:00,90,0.0,No Info,2050
2,Jet Airways,2019-06-09,Delhi,Cochin,18:25:00,04:25:00,600,2.0,No Info,14237
3,Jet Airways,2019-05-09,Delhi,Cochin,18:15:00,12:35:00,1100,1.0,In-flight meal not included,12373
4,Goair,2019-06-24,Banglore,Delhi,07:45:00,10:40:00,175,0.0,No Info,3898


In [65]:
#export_data(X_test,y_test,"test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Spicejet,2019-05-15,Banglore,Delhi,21:10:00,00:05:00,175,0.0,No Info,3625
1,Air India,2019-03-21,Delhi,Cochin,14:10:00,17:10:00,180,0.0,No Info,5674
2,Jet Airways,2019-06-03,Delhi,Cochin,19:45:00,12:35:00,1010,1.0,No Info,14714
3,Vistara,2019-04-01,Delhi,Cochin,14:40:00,17:50:00,190,0.0,No Info,4851
4,Jet Airways,2019-03-21,Delhi,Cochin,13:00:00,18:50:00,350,1.0,In-flight meal not included,6540


In [66]:
#first we split the test set out, and then using other set we split train and validation
#Validation set is to tune the parameters
#then train again on training set